In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
from loguru import logger
logger.remove()
logger.add(sys.stdout, level="INFO")
# logger.remove()
# logger.add(sys.stdout, level="DEBUG")

1

In [3]:
import sys
import os
sys.path.insert(0, '../')

from llmagent.privatestore import WebStore, WebConf
from llmagent.secret import AK_SK
from llmagent.llmapi import model_spec, QwenEmbedding, QwenLLM
qw_key = AK_SK("../llmagent/secret/keystore/qwen.keys")
os.environ["OPENAI_API_KEY"] = qw_key.ak
llm = QwenLLM(model_spec=model_spec.QwenPlus, temperature=0.6)
embedding = QwenEmbedding(api_key=qw_key.ak)
webstore = WebStore("llmtutorial", "../vectorstore/llmtutorial_qwen", emb_func=embedding, extra_config=WebConf(
    root_url="https://aitutor.liduos.com/",
    limit=10,
))

USER_AGENT environment variable not set, consider setting it to identify your requests.
[WARNING][2024-10-26 22:55:10.839] redis_rate_limiter.py:21 [t:8573500096]: no redis installed, RedisRateLimiter unavailable


In [6]:
# webstore.reload()

/Users/flybywindwen/miniconda3/envs/llm/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aitutor.liduos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/flybywindwen/miniconda3/envs/llm/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aitutor.liduos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/flybywindwen/miniconda3/envs/llm/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aitutor.liduos.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#t

2024-10-26 21:54:47.687 | INFO     | llmagent.privatestore.base:__load__:51 - Adding documents 0 to 256
2024-10-26 21:55:04.689 | INFO     | llmagent.privatestore.base:__load__:51 - Adding documents 256 to 512
2024-10-26 21:55:20.084 | INFO     | llmagent.privatestore.base:__load__:51 - Adding documents 512 to 768
2024-10-26 21:55:32.399 | INFO     | llmagent.privatestore.base:__load__:51 - Adding documents 768 to 1024
2024-10-26 21:55:46.842 | INFO     | llmagent.privatestore.base:__load__:51 - Adding documents 1024 to 1280
2024-10-26 21:56:03.868 | INFO     | llmagent.privatestore.base:__load__:51 - Adding documents 1280 to 1536


In [5]:
webstore.collection_count()

1393

In [4]:
webstore._documents['https://aitutor.liduos.com/01-llm/01-2.html'].page_content

'作者：莫尔索 (https://liduos.com/about)     前言 (../)    关注《莫尔索随笔》 (https://liduos.com/wxqcode.png)    大语言模型概述    大语言模型概况 (01-1.html)    你好, ChatGPT (01-2.html)    OpenAI 文档解读 (01-3.html)    动手实现聊天机器人 (01-4.html)    基于 OpenAI API 搭建一个端到端问答系统 (01-5.html)    LLM 安全专题 (01-6.html)      LangChain入门    LangChain介绍 (../02-langchain/02-1.html)    LangChain模块学习 (../02-langchain/02-2.html)    LangChain之Chains模块 (../02-langchain/02-2-1.html)    LangChain之Agents模块 (../02-langchain/02-2-2.html)    LangChain之Callback模块 (../02-langchain/02-2-3.html)    Embedding嵌入 (../02-langchain/02-3.html)    动手实现文档问答机器人 (../02-langchain/02-4.html)      LlamaIndex 概述    LlamaIndex介绍 (../03-llamaIndex/03-1.html)    LlamaIndex索引 (../03-llamaIndex/03-2.html)    动手实现企业知识库 (../03-llamaIndex/03-3.html)      HuggingGPT 实现    HuggingFace 介绍 (../04-huggingface/04-1.html)    transformers 库基础组件 (../04-huggingface/04-2.html)    多模态任务设计 (../04-huggingface/04-3.html)    动手实现 HuggingGPT (../04-huggingface/04-4.html)      LLMOps 专题    L

## 构建Prompt

In [5]:
from llmagent.chain import QAWithContextChain, TemplateConf
prompt = QAWithContextChain(llm=llm, 
                               vectorstore=webstore.vectorstore(),
                               conf=TemplateConf(context_num=10, history_num=3))

In [6]:
import time
ans = prompt.stream("langchain提示词管理")
for token in ans:
    for c in token:
        print(c, end="")
    

在LangChain中，提示词管理是一项重要的功能，它帮助确保与大语言模型（LLMs）交互时的输入能够灵活且结构化。以下是一些关键点：

-**提示模板**：这是创建动态提示词的基础。通过定义一个包含变量的模板，可以在运行时插入具体的值，从而生成个性化的提示词。例如，`PromptTemplate`类允许你指定`input_variables`和 `template`字段来构建这样的模板。```pythonfrom langchain importPromptTemplate# 没有输入变量的简单提示no_input_prompt =PromptTemplate(input_variables=[], template="Tellme a joke.")print(no_input_prompt.format())# 包含一个输入变量的提示
  one_input_prompt = PromptTemplate(input_variables=["adjective"], template="Tell me a{adjective}joke.")
  print(one_input_prompt.format(adjective="funny"))
  ```

- **动态提示词**：结合提示模板和变量，可以实现高度定制化的提示词生成。这不仅提高了灵活性，还保证了提示词内容的结构一致性。

- **少样本提示模板（FewShotPromptTemplate）**：这种模板用于提供少量示例给模型，以便模型能够在特定上下文中更好地理解任务要求。

- **特征存储关联（FeaturePromptTemplate）**：将提示词与特征存储相结合，可以从数据源动态提取提示词，适用于需要实时更新提示内容的场景。

- **防御性措施**：由于提示注入攻击的存在，合理地管理和验证提示词是非常必要的。可以通过设置默认值、限制输入长度等方式减少潜在风险。

更多关于如何设计有效提示词的信息，可以参考[Prompt专题](https://aitutor.liduos.com/ref/prompt.html)。此外，为了深入了解提示工程的最佳实践，推荐查阅 [Prompt EngineeringGuide](https://www.promptingguide.ai/zh)和 [Le

In [7]:
ans = prompt.stream("langchain回调管理器如何使用")
for token in ans:
    for c in token:
        print(c, end="")

在LangChain中，回调管理器（Callback Manager）是用于处理来自大语言模型（LLM）应用程序各个阶段的事件的重要组件。它支持日志记录、监控、流式处理等功能，有助于解决LLM的幻觉问题。下面是使用回调管理器的基本步骤：

1. **定义回调处理器**：首先，你需要定义一个或多个回调处理器。这些处理器继承自`BaseCallbackHandler`基类，并重写其方法以实现特定的功能。例如，你可以创建一个用于记录日志的回调处理器。

   ```pythonfrom langchain.callbacks.base importBaseCallbackHandlerclass MyLogger(BaseCallbackHandler):def on_llm_start(self,serialized: Dict[str, Any], prompts: List[str],**kwargs: Any) -> Any:print("LLM started with prompts:", prompts)```

2.**创建回调管理器实例**：接下来，创建一个回调管理器实例，并将你的回调处理器添加进去。LangChain提供了同步和异步两种类型的回调管理器，分别是`CallbackManager`和`AsyncCallbackManager`。

   ```pythonfrom langchain.callbacks.manager importCallbackManagercallback_manager = CallbackManager([MyLogger()])
   ```3. **在链或模型中使用回调管理器**：最后，在创建链（Chain）或调用语言模型时，将回调管理器传递给它们。这样，当执行相应的操作时，就会触发你定义的回调函数。```pythonfrom langchain.chains.base importChain
   fromlangchain.prompts.base import BasePromptTemplate
   fromlangchain.llmsimport OpenAIclass MyCustomChain(Chain):prompt: BasePromptTemplatellm: BaseLanguageM

In [38]:
prompt.conf.history_num=2

In [9]:
prompt.get_history_wo_role()

'langchain提示词管理\nlangchain回调管理器如何使用'

In [10]:
ans = prompt.stream("除了langchain，还有哪些ChatGPT框架")
for token in ans:
    for c in token:
        print(c, end="")

除了LangChain之外，还有一些其他的ChatGPT框架和工具，它们各自具有不同的特点和优势：

- **Dust.tt**：这是一个用Rust和TypeScript编写的框架，提供了简单易用的API，使开发者能够快速构建自己的LLM应用程序。它的优点在于易于上手和集成。

- **其他竞品**：虽然没有具体列出名称，但市场上确实存在多个类似的工具框架，它们可能在某些方面与LangChain形成竞争，比如提供更强大的性能评估功能、更好的社区支持或者更加丰富的插件生态系统等。

如果您对某个特定框架感兴趣，建议查看其官方文档或GitHub页面获取更多信息。source: https://aitutor.liduos.com/02-langchain/02-1.html

In [39]:
prompt.get_history_wo_role()

'langchain回调管理器如何使用\n除了langchain，还有哪些ChatGPT框架'